In [1]:
# !wget https://huggingface.co/datasets/malaysia-ai/starcoderdata-sample/resolve/main/dockerfile.jsonl

In [2]:
from huggingface_hub import InferenceClient
from tqdm import tqdm
import os
import json
from glob import glob

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
ls = []

with open('dockerfile.jsonl') as fopen:
    for l in tqdm(fopen):
        try:
            l = json.loads(l)
            if len(l['content'].split()) > 5e3:
                continue
            ls.append(l['content'])
        except:
            pass
len(ls)

100000it [00:01, 74477.98it/s]


99998

In [4]:
client = InferenceClient(
    '', timeout = 120
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

def format_user(history):
    prompt = "<s>"
    for user_prompt, bot_response in history:
        prompt += f"[INST] {user_prompt} [/INST]"
        prompt += f" {bot_response}</s> "
    prompt += f"[INST]"
    return prompt

In [5]:
generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=5120,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
)

In [6]:
!mkdir mixtral-dockerfile
# !rm -rf mixtral-dockerfile/*.json

mkdir: cannot create directory ‘mixtral-dockerfile’: File exists


In [7]:
def answer(q, i):
    filename = f'mixtral-dockerfile/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
    # for _ in range(5):
        try:
            
            prompt = f"""
Please gain inspiration from the following random code snippet to create a high-quality Dockerfile problem. Present your output in two distinct sections: [Problem Description] and [Solution].

Dockerfile snippet for inspiration:
```
{q}
```

Guidelines for each section:

1. [Problem Description]: This should be **completely self-contained**, providing all the contextual information one needs to understand and solve the problem. Assume common programming knowledge, but ensure that any specific context, variables, or code snippets pertinent to this problem are explicitly included but do not mention code snippet above.

2. [Solution]: Offer a comprehensive, **correct** solution that accurately addresses the [Problem Description] you provided.
""".strip()
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump(output, fopen)
            break
        except Exception as e:
            if 'tokens + `max_new_tokens`' in str(e):
                with open(filename, 'w') as fopen:
                    json.dump(False, fopen)
                break
            pass

In [8]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [9]:
from threading import Thread
from queue import Queue

queue = Queue()
questions = ls
urls = [(q, no) for no, q in enumerate(questions)]
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [10]:
answer(*urls[3])

In [11]:
with open('mixtral-dockerfile/3.json') as fopen:
    print(json.loads(fopen.read()))

 **[Problem Description]**

The task is to create a Dockerfile that will build an image for running Hubble, a network traffic analyzer and visualizer. Hubble requires Python 3.7, pip, git, osquery, and vcredist prerequisites to be installed in the environment. Additionally, it needs several packages installed via pip. The resulting image should have all the necessary dependencies and be ready to run Hubble.

The Dockerfile should meet the following requirements:

1. Based on a lightweight Windows base image.
2. Install Python 3.7, pip, git, osquery, and vcredist prerequisites.
3. Clone the Hubble repository and checkout the specified version.
4. Build Hubble and its dependencies using pip.
5. Set up an entry point to run Hubble when the container starts.

**[Solution]**

Create a `Dockerfile` with the following content:

```Dockerfile
# Use a lightweight Windows base image
FROM mcr.microsoft.com/windows/servercore:ltsc2019

# Set environment variables
ENV PYTHONIOENCODING='UTF-8' \
   

In [ ]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

 17%|████████████████▋                                                                                  | 16819/99998 [6:17:02<18:18:07,  1.26it/s]